In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install git+git://github.com/albertbup/deep-belief-network.git

  Cloning git://github.com/albertbup/deep-belief-network.git to /tmp/pip-req-build-ygjbsiod
  Running command git clone -q git://github.com/albertbup/deep-belief-network.git /tmp/pip-req-build-ygjbsiod
  Created wheel for deep-belief-network: filename=deep_belief_network-1.0.3-cp36-none-any.whl size=13464 sha256=6ae23d42365090150ec9e1a1754a6f61bfdb9d39d96462ca510bb4387d11078d
  Stored in directory: /tmp/pip-ephem-wheel-cache-m8sb3q2z/wheels/29/6d/3b/6a50cf42a32bdfaa903b17832d60d8d3e5dc4b0fd02ae8acaf
Successfully built deep-belief-network


In [ ]:
!pip install git+git://github.com/albertbup/deep-belief-network.git@master_gpu

  Cloning git://github.com/albertbup/deep-belief-network.git (to revision master_gpu) to /tmp/pip-req-build-gg8icvlh
  Running command git clone -q git://github.com/albertbup/deep-belief-network.git /tmp/pip-req-build-gg8icvlh
  Running command git checkout -b master_gpu --track origin/master_gpu
  Switched to a new branch 'master_gpu'
  Branch 'master_gpu' set up to track remote branch 'master_gpu' from 'origin'.
     |████████████████████████████████| 394.7MB 40kB/s 
  Created wheel for deep-belief-network: filename=deep_belief_network-1.0.3-cp36-none-any.whl size=13465 sha256=a982e7de041e70005be20fc02e6d49a9301052fc2147a567f5a4d70a64c44fed
  Stored in directory: /tmp/pip-ephem-wheel-cache-p74vndjm/wheels/fe/5a/c7/18fc337ca5590a8b7c74c17ddba7a63935ce7a5bafb6cdec02
Successfully built deep-belief-network


In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd

np.random.seed(1337)  # for reproducibility
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from sklearn.metrics.classification import accuracy_score
tf.compat.v1.Session()
from dbn import SupervisedDBNClassification
#from dbn.tensorflow import SupervisedDBNClassification
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, Normalizer
from keras.utils.np_utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error,roc_auc_score,cohen_kappa_score)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
##########  DBN  ###########

# data is being refined, Here we fist get our data to be converted into pandas dataframae
# then we drop empty spaces and eradicate useless indices and getting data in float type
def cleaningdata(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
# fetching dataset from path
dataset = pd.read_csv('/content/drive/MyDrive/IoT Intrusion Dataset 2020.csv', encoding='utf-8')
# for object type data we will label it and transform into apporopriate type fo data after using fit_transform on that colomn to avoid errors
for column in dataset.columns:
    if dataset[column].dtype == type(object):
        le = LabelEncoder()
        dataset[column] = le.fit_transform(dataset[column])
data = cleaningdata(dataset)

# collecting all required colomns in data to X
X = data.iloc[:, 0:85].values

# collecting our LABEL colomn in y
y = data.iloc[:, -2].values
# splitting the data on ratio of 20% test data and 80% data from our encoded X and Y and getting our test and train data parameter
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=52)

sc = Normalizer()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train.shape

(500332, 85)

In [ ]:
# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=90,
                                         batch_size=32,
                                         activation_function='relu',
                                         dropout_p=0.2)
classifier.fit(X_train, y_train)

# Save the model
classifier.save('model.pkl')

# Restore it
classifier = SupervisedDBNClassification.load('model.pkl')


[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 0.013197
>> Epoch 2 finished 	RBM Reconstruction error 0.011507
>> Epoch 3 finished 	RBM Reconstruction error 0.011458
>> Epoch 4 finished 	RBM Reconstruction error 0.010850
>> Epoch 5 finished 	RBM Reconstruction error 0.010567
>> Epoch 6 finished 	RBM Reconstruction error 0.010787
>> Epoch 7 finished 	RBM Reconstruction error 0.010900
>> Epoch 8 finished 	RBM Reconstruction error 0.010601
>> Epoch 9 finished 	RBM Reconstruction error 0.010732
>> Epoch 10 finished 	RBM Reconstruction error 0.010670
>> Epoch 1 finished 	RBM Reconstruction error 0.011070
>> Epoch 2 finished 	RBM Reconstruction error 0.008505
>> Epoch 3 finished 	RBM Reconstruction error 0.007563
>> Epoch 4 finished 	RBM Reconstruction error 0.006818
>> Epoch 5 finished 	RBM Reconstruction error 0.006469
>> Epoch 6 finished 	RBM Reconstruction error 0.006266
>> Epoch 7 finished 	RBM Reconstruction error 0.006330
>> Epoch 8 finished 	RBM Reconstructi

In [ ]:
### train_accuracy
import sklearn.metrics as metrics
target_names = ['Mirai', 'DoS', 'Scan', 'Normal', 'MITM_ARP_Spoofing']
expected = y_train
predicted = classifier.predict(X_train)
data = classification_report(expected,np.round(predicted), target_names=target_names)
print(data)
conmat = confusion_matrix(expected,predicted)
print(conmat)

print("Accuracy_Train:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision_Train:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall_Train:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score_Train:",metrics.f1_score(expected, np.round(predicted), average='weighted'))


                   precision    recall  f1-score   support

            Mirai       1.00      1.00      1.00     47576
              DoS       0.99      0.36      0.52     28268
             Scan       0.82      1.00      0.90    332252
           Normal       0.95      0.46      0.62     32013
MITM_ARP_Spoofing       0.97      0.34      0.51     60223

         accuracy                           0.85    500332
        macro avg       0.94      0.63      0.71    500332
     weighted avg       0.87      0.85      0.82    500332

[[ 47424      3    134     14      1]
 [     0  10091  18153     23      1]
 [     0    102 331439    518    193]
 [     6      1  16892  14577    537]
 [     0      1  39252    197  20773]]
Accuracy_Train: 0.8480448981876034
Precision_Train: 0.8703791493209396
Recall_Train: 0.8480448981876034
f1-score_Train: 0.8215320506943216


In [ ]:
### test_accuracy
target_names = ['Mirai', 'DoS', 'Scan', 'Normal', 'MITM_ARP_Spoofing']
expected = y_test
predicted = classifier.predict(X_test)

print("Accuracy:",metrics.accuracy_score(expected, np.round(predicted)))
print("Precision:",metrics.precision_score(expected, np.round(predicted), average='weighted'))
print("Recall:",metrics.recall_score(expected, np.round(predicted), average='weighted'))
print("f1-score:",metrics.f1_score(expected, np.round(predicted), average='weighted'))
#dt classification report
print(classification_report(expected, np.round(predicted), target_names=target_names))

Accuracy: 0.8474053228656172
Precision: 0.8690220158012448
Recall: 0.8474053228656172
f1-score: 0.8209428873149593
                   precision    recall  f1-score   support

            Mirai       1.00      1.00      1.00     11815
              DoS       0.98      0.36      0.53      7109
             Scan       0.82      1.00      0.90     83057
           Normal       0.95      0.46      0.62      8060
MITM_ARP_Spoofing       0.96      0.34      0.51     15042

         accuracy                           0.85    125083
        macro avg       0.94      0.63      0.71    125083
     weighted avg       0.87      0.85      0.82    125083

